In [1]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from IPython.display import clear_output, display
from selenium.webdriver.common.by import By
from selenium import webdriver
from time import sleep
from tqdm import tqdm
import pandas as pd
import re

In [2]:
df = pd.read_csv("links_2.csv")
df

,youtube_link,caption,download_link,success,downloaded
0,https://youtu.be/QvI4-SJIMl4,1__آشنایی_با_تانسور_و_ریاضیات_تانسوری_جلسه_۱_ت...,https://srv3.dlmate.cc/dl?hash=PvEVv5qLsx9FH88...,True,NaN
1,https://youtu.be/wAxgvfOk_XI,2__آشنایی_با_تانسور_و_ریاضیات_تانسوری_جلسه_۲_ق...,https://srv5.dlmate.cc/dl?hash=Y0Ymh2Pm1Uh%2FR...,True,NaN
2,https://youtu.be/iXtM56jFuCw,NaN,NaN,NaN,NaN
3,https://youtu.be/8D4UWuzkG28,NaN,NaN,NaN,NaN
4,https://youtu.be/NFO1IqNrjIc,NaN,NaN,NaN,NaN
5,https://youtu.be/e3xGddJ-HeQ,NaN,NaN,NaN,NaN
6,https://youtu.be/kry9gbr0w9M,NaN,NaN,NaN,NaN
7,https://youtu.be/jDgdEayWLnQ,NaN,NaN,NaN,NaN
8,https://youtu.be/M8B0GlUhZfw,NaN,NaN,NaN,NaN


In [3]:
def sanitize_filename(caption):
    caption = re.sub(r'Duration:\s*\d{1,2}:\d{2}\s*minutes$', '', caption, flags=re.MULTILINE)
    filename = re.sub(r'[^\w\s-]', '', caption)
    filename = re.sub(r'[\s\n]+', '_', filename)
    filename = filename.strip('_')
    return filename[:240]

In [4]:
def save_download_video(youtube_link):
    try:
        url = 'https://en.y2mate.is/v96/'

        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')

        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

        # driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

        driver.get(url)

        driver.maximize_window()

        a = driver.find_elements(By.ID, "txtUrl")[0].send_keys(youtube_link)

        b = driver.find_elements(By.ID, "btnSubmit")[0].click()

        sleep(3)

        trs = driver.find_elements(By.TAG_NAME, 'tr')

        for i in trs:
            if "360p" in i.text or "480p" in i.text or "720p" in i.text:
                tr = i
                break

        tr.find_element(By.TAG_NAME, 'button').click()

        sleep(3)

        count = 0

        while True:

            count += 1

            if count > 20:
                break

            try:
                download_btn = driver.find_element(By.ID, "btnOk")
                break

            except:
                sleep(3)
                continue

        x = download_btn.find_element(By.TAG_NAME, "button").find_element(By.TAG_NAME, "a")

        sleep(3)

        download_link = x.get_attribute("href")

        caption = driver.find_element(By.TAG_NAME, "b").text

        caption = sanitize_filename(caption)

        driver.quit()

        return download_link, caption
    
    except Exception as e:
        print(e)
        return None, None

In [5]:
for index, row in tqdm(df.iterrows()):

    clear_output(wait=True)
    display(f"INDEX : {index}")

    if pd.notna(row['download_link']):
        continue

    download_link, caption = save_download_video(row['youtube_link'])

    caption = f"{index + 1}__{caption}"

    df.at[index, 'download_link'] = download_link
    df.at[index, 'caption'] = caption

    if download_link and caption:
        df.at[index, 'success'] = True

    df.to_csv("links_2.csv", index=False)

'INDEX : 8'

9it [01:24,  9.43s/it]
